# Fetching Stock Prices

In [67]:
import pandas as pd
import yfinance as yf
import datetime
from joblib import Parallel, delayed
import multiprocessing
import json 
import json.decoder
import os 
import requests
import pandas_datareader
import alpha_vantage.timeseries

In [79]:
ts = alpha_vantage.timeseries.TimeSeries(key='QVEGLYRJUK0ABVMT',output_format='pandas')
data, meta_data = ts.get_intraday('GOOGL','1min','full')

In [80]:
data

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2023-01-12 20:00:00,91.12,91.21,91.12,91.21,842.0
2023-01-12 19:59:00,91.14,91.15,91.14,91.15,568.0
2023-01-12 19:58:00,91.12,91.12,91.12,91.12,428.0
2023-01-12 19:56:00,91.16,91.16,91.16,91.16,379.0
2023-01-12 19:52:00,91.16,91.19,91.16,91.19,1129.0
...,...,...,...,...,...
2022-12-30 04:11:00,87.66,87.67,87.66,87.67,497.0
2022-12-30 04:08:00,87.65,87.69,87.65,87.69,1689.0
2022-12-30 04:05:00,87.49,87.55,87.49,87.50,1330.0


In [56]:
env_json = {
    'Current_day': 1
}
with open('env.json',"w") as outfile:
    json.dump(env_json,outfile)

In [57]:
tmp_stocks = pd.read_csv('nasdaq.csv')

In [58]:
symbols = list(tmp_stocks['Symbol'])

In [59]:
tmp_json = {}
counter = 0 
sub_count = 0 
for idx, i in enumerate(symbols):
    if "^" in i: 
        continue 
    if sub_count % 500 == 0:
        counter += 1 
        tmp_json[counter] = []
    tmp_json[counter].append(i)
    sub_count += 1 

In [60]:
len(tmp_json)

14

In [61]:
with open("stocks.json","w") as outfile:
    json.dump(tmp_json,outfile)

In [63]:
pandas_datareader.data.DataReader('AAIC' , 'av-intraday', start=datetime.datetime(2022,1,1),end=datetime.datetime(2023,1,14),api_key='QVEGLYRJUK0ABVMT',)

,open,high,low,close,volume
2022-12-30 09:31:00,2.9000,2.9100,2.9000,2.9100,255
2022-12-30 09:32:00,2.9100,2.9100,2.9100,2.9100,1000
2022-12-30 09:34:00,2.9101,2.9101,2.9101,2.9101,1000
2022-12-30 09:35:00,2.9200,2.9200,2.9200,2.9200,517
2022-12-30 09:45:00,2.9200,2.9200,2.9200,2.9200,250
...,...,...,...,...,...
2023-01-12 15:56:00,3.0000,3.0000,3.0000,3.0000,1735
2023-01-12 15:57:00,3.0000,3.0000,3.0000,3.0000,643
2023-01-12 15:58:00,3.0000,3.0000,2.9998,2.9998,8280
2023-01-12 15:59:00,3.0100,3.0100,3.0100,3.0100,620
